In [17]:
import pandas as pd
import numpy as np

# Project Goal: predict whether it's recurring event -- 1 or 0

## Step 1: understanding features

### training: info of 1st observable event for each unique user-song pair 

In [18]:
train = pd.read_csv("train.csv")

In [4]:
train.shape

(7377418, 6)

In [5]:
train.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1


In [6]:
# percentage missing is ok
train.isnull().sum()/train.shape[0]*100

msno                  0.000000
song_id               0.000000
source_system_tab     0.336825
source_screen_name    5.622618
source_type           0.291959
target                0.000000
dtype: float64

In [7]:
train.msno.nunique()

30755

In [8]:
train.song_id.nunique()

359966

In [10]:
train.source_system_tab.value_counts()

my library      3684730
discover        2179252
search           623286
radio            476701
listen with      212266
explore          167949
notification       6185
settings           2200
Name: source_system_tab, dtype: int64

In [9]:
train.source_screen_name.value_counts()

Local playlist more     3228202
Online playlist more    1294689
Radio                    474467
Album more               420156
Search                   298487
Artist more              252429
Discover Feature         244246
Discover Chart           213658
Others profile more      201795
Discover Genre            82202
My library                75980
Explore                   72342
Unknown                   54170
Discover New              15955
Search Trends             13632
Search Home               13482
My library_Search          6451
Self profile more           212
Concert                      47
Payment                      12
Name: source_screen_name, dtype: int64

In [11]:
train.source_type.value_counts()

local-library             2261399
online-playlist           1967924
local-playlist            1079503
radio                      483109
album                      477344
top-hits-for-artist        423614
song                       244722
song-based-playlist        210527
listen-with                192842
topic-article-playlist      11194
artist                       3038
my-daily-playlist             663
Name: source_type, dtype: int64

In [13]:
train.target.value_counts() # pretty balanced data set? 

1    3714656
0    3662762
Name: target, dtype: int64

### testing: 
#### For each id in the test set, you must predict a probability for the target variable. The file should contain a header and have the following format:
- id,target
- 2,0.3
- 5,0.1
- 6,1
- etc.

#### Submissions are evaluated on area under the ROC curve between the predicted probability and the observed target.

In [19]:
test = pd.read_csv("test.csv")

In [21]:
test.shape

(2556790, 6)

In [23]:
test.head()

,id,msno,song_id,source_system_tab,source_screen_name,source_type
0,0,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,my library,Local playlist more,local-library
1,1,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,y/rsZ9DC7FwK5F2PK2D5mj+aOBUJAjuu3dZ14NgE0vM=,my library,Local playlist more,local-library
2,2,/uQAlrAkaczV+nWCd2sPF2ekvXPRipV7q0l+gbLuxjw=,8eZLFOdGVdXBSqoAv5nsLigeH2BvKXzTQYtUM53I0k4=,discover,NaN,song-based-playlist
3,3,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,ztCf8thYsS4YN3GcIL/bvoxLm/T5mYBVKOO4C9NiVfQ=,radio,Radio,radio
4,4,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,MKVMpslKcQhMaFEgcEQhEfi5+RZhMYlU3eRDpySrH8Y=,radio,Radio,radio


In [36]:
# percentage missing is ok
test.isnull().sum()/test.shape[0]*100

id                    0.000000
msno                  0.000000
song_id               0.000000
source_system_tab     0.330180
source_screen_name    6.370605
source_type           0.285397
dtype: float64

### sample submission

In [25]:
submit = pd.read_csv("sample_submission.csv")

In [26]:
submit.shape

(2556790, 2)

In [27]:
submit.head()

,id,target
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


## Understanding songs features

### songs: data in unicode

In [20]:
song = pd.read_csv("songs.csv")

In [15]:
song.shape

(2296320, 9)

In [16]:
song.head()

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language,name,isrc
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0,焚情,TWB531410010
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0,PLAYING WITH FIRE,NaN
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0,SORRY| SORRY,NaN
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0,愛我的資格,TWC950206108
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0,Mary Had a Little Lamb,NaN


In [38]:
song.isnull().sum()/song.shape[0]*100

song_id         0.000000
song_length     0.000000
genre_ids       4.098558
artist_name     0.000000
composer       46.655257
lyricist       84.712409
language        0.000044
dtype: float64

##### since composer and lyricist have a lot missing --> need a way to handle this

### song_extra_info

In [21]:
song2 = pd.read_csv("song_extra_info.csv")

In [44]:
song2.shape

(2295971, 3)

In [45]:
song2.head()

,song_id,name,isrc
0,LP7pLJoJFBvyuUwvu+oLzjT+bI+UeBPURCecJsX1jjs=,我們,TWUM71200043
1,ClazTFnk6r0Bnuie44bocdNMM3rdlrq0bCGAsGUWcHE=,Let Me Love You,QMZSY1600015
2,u2ja/bZE3zhCGxvbbOB3zOoUjx27u40cf5g09UXMoKQ=,原諒我,TWA530887303
3,92Fqsy0+p6+RHe2EoLKjHahORHR1Kq1TBJoClW9v+Ts=,Classic,USSM11301446
4,0QFmz/+rJy1Q56C1DuYqT9hKKqi5TUqx0sN0IwvoHrw=,愛投羅網,TWA471306001


In [46]:
song2.isnull().sum()/song2.shape[0]*100

song_id    0.000000
name       0.000087
isrc       5.947288
dtype: float64

In [47]:
# Multiple songs could share one ISRC since a single recording could be re-published several times.

### combining song info

In [ ]:
song.song_id

In [50]:
ind = song2.song_id.isin(song.song_id) #& df1.user_id.isin(df2.user_id)

In [52]:
ind.shape # all song2 are included in song, so just merge the two dataframe

(2295971,)

In [26]:
songs = pd.merge(song, song2, how='left', on=['song_id'])

In [54]:
songs.shape

(2296320, 9)

In [55]:
songs.head()

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language,name,isrc
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0,焚情,TWB531410010
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0,PLAYING WITH FIRE,NaN
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0,SORRY| SORRY,NaN
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0,愛我的資格,TWC950206108
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0,Mary Had a Little Lamb,NaN


In [95]:
songs.to_csv("songs.csv", index=False)

In [5]:
songs = pd.read_csv("songs.csv")

In [6]:
songs.isnull().sum()/songs.shape[0]*100

song_id         0.000000
song_length     0.000000
genre_ids       4.098558
artist_name     0.000000
composer       46.655257
lyricist       84.712409
language        0.000044
name            0.039193
isrc            5.984706
dtype: float64

In [7]:
songs.artist_name

0                                           張信哲 (Jeff Chang)
1                                                  BLACKPINK
2                                               SUPER JUNIOR
3                                                      S.H.E
4                                                       貴族精選
5                                                       貴族精選
6                                            伍佰 & China Blue
7                                          光良 (Michael Wong)
8                                               林俊傑 (JJ Lin)
9                                                   Kodaline
10                                       D.L 羅時豐 (Daniel Lo)
11                                                  白安 (Ann)
12                                                Littlesong
13                                             蔡旻佑 (Evan Yo)
14                                                  Coldplay
15                                             Maggie Rogers
16                      

### how many songs are in the training dataset: 
- check week1 meeting notes for image representation

In [84]:
s1 = pd.merge(songs, train, how='inner', on=['song_id'])

In [85]:
s1.shape# all are included

(7377304, 14)

In [89]:
s1.song_id.nunique()

359914

In [71]:
train.song_id.nunique()

359966

In [72]:
songs.song_id.nunique()

2296320

In [92]:
ss = songs[~songs.song_id.isin(train.song_id)]

In [93]:
ss.song_id.nunique()

1936406

In [94]:
1936406+359914

2296320

### how many songs are in the testing dataset:

In [97]:
s2 = pd.merge(songs, test, how='inner', on=['song_id'])
s2.shape

(2556765, 14)

In [104]:
s2.song_id.nunique()

224739

In [100]:
test.song_id.nunique()

224753

In [103]:
songs.song_id.nunique()

2296320

In [105]:
53-39

14

In [106]:
2296320-224739

2071581

### members: user information

In [22]:
member = pd.read_csv("members.csv")

In [40]:
member.shape

(34403, 7)

In [41]:
member.head()

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,NaN,7,20110820,20170920
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,NaN,7,20150628,20170622
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,NaN,4,20160411,20170712
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,NaN,9,20150906,20150907
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,NaN,4,20170126,20170613


In [42]:
member.isnull().sum()/member.shape[0]*100

msno                       0.000000
city                       0.000000
bd                         0.000000
gender                    57.849606
registered_via             0.000000
registration_init_time     0.000000
expiration_date            0.000000
dtype: float64

# merging dataframes

In [27]:
df_train_members = pd.merge(train, member, on='msno', how='inner')
df_train = pd.merge(df_train_members, songs, on='song_id', how='outer')

In [28]:
df_test_members = pd.merge(test, member, on='msno', how='inner')
df_test = pd.merge(df_test_members, songs, on='song_id', how='outer')

In [32]:
df_train= df_train[pd.notnull(df_train['msno'])]
df_test = df_test[pd.notnull(df_test['msno'])]

In [35]:
df_train.shape

(7377418, 22)

In [36]:
df_test.shape

(2556790, 22)

In [37]:
train.shape

(7377418, 6)

In [38]:
df_train.to_csv("df_train.csv", index = False)


In [39]:
df_test.to_csv("df_test.csv", index = False)

## saving the target and the id columns separately

In [40]:
df_train_target = df_train['target'].astype(np.int8)
df_test_id = df_test['id']

df_train_target.to_csv("df_train_target.csv", index = False)
df_test_id.to_csv("df_test_id.csv", index = False)

In [41]:
df_train.drop('target', axis=1, inplace=True)
df_test.drop('id', axis=1, inplace=True)

/Users/yanzeng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [42]:
df_train.to_csv("df_train_new.csv", index = False)

In [43]:
df_test.to_csv("df_test_new.csv", index = False)